# Data Cleaning for Float Columns

**Basic imports**

In [27]:
%matplotlib inline
import numpy as np
import numpy.random as nd
import pandas as pd
import math
import matplotlib.pyplot as plt

import os
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from IPython.display import display


from collections import Counter

**Additional Imports**

In [48]:
from IPython.display import display, HTML

**Load the data**

In [28]:
inputFile='../../data/smallData/tmp/AcceptedLoans.csv'
data=pd.read_csv(inputFile)
data.head()

/Users/joshfeldman/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


**Remove Columns that are entirely empty**

In [30]:
data = data.loc[:, (data.isna().sum(axis=0) != len(data))]
data.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


**Check Data Types**

In [32]:
types = data.dtypes
types

id                             object
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
desc                           object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
mths_since_l

**Get Float Columns**

In [33]:
float_cols = types[types == 'float64']
float_cols

loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
installment                   float64
annual_inc                    float64
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
mths_since_last_delinq        float64
mths_since_last_record        float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
collections_12_mths_ex_med    float64
policy_code                   float64
acc_now_deli

## CLEAN THE FLOAT FEATURES

To clean the float features we will perform the following steps:
1. Make sure missing values are `None` by examining rows where the data is 0
2. Examine edge cases by looking at lowest 20 rows and highest 20 rows
3. Check to see if data should be categorical by looking at number of unique values

In [72]:
def examine(col_name, df):
    iszero = df[col_name] == 0
    islow = df[col_name] <= df[col_name].quantile(.01)
    ishigh = df[col_name] >= df[col_name].quantile(.99)
    print("rows where ")
    display(df[iszero])
    display(df[islow])
    display(df[ishigh])


In [71]:
col = float_cols.index[0]
print('column:', col)
examine(col, data)

column: loan_amnt


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
15,NaN,1000.0,1000.0,1000.000000,36 months,16.29%,35.31,D,D1,Internal revenue Service,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
422,NaN,1000.0,1000.0,1000.000000,36 months,14.65%,34.50,C,C3,NaN,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
801,NaN,1000.0,1000.0,1000.000000,36 months,13.49%,33.94,C,C1,Inviting Company,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
876,NaN,1200.0,1200.0,1200.000000,36 months,12.69%,40.26,B,B5,san gabriel transit inc,...,0.0,N,Cash,Y,Jun-17,COMPLETE,Jul-16,697.92,65.0,0.0
1225,NaN,1200.0,1200.0,1200.000000,36 months,7.90%,37.55,A,A4,"City of Tulsa, Okla",...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1394,NaN,1000.0,1000.0,1000.000000,36 months,13.49%,33.94,C,C1,MicroStrategy,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1460,NaN,1000.0,1000.0,1000.000000,36 months,15.27%,34.80,C,C4,General Motors,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1536,NaN,1200.0,1200.0,1200.000000,36 months,13.49%,40.72,C,C1,"APM Terminals, NA",...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,1000.0,1000.0,1000.000000,36 months,10.65%,32.58,B,B2,ShopRite,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1837,NaN,1000.0,1000.0,1000.000000,36 months,19.03%,36.68,E,E2,"AJ Industries West, Inc.",...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
80,NaN,35000.0,22075.0,22050.00000,60 months,17.27%,551.84,D,D3,tutor Per ini Corporation,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
128,NaN,35000.0,35000.0,35000.00000,36 months,8.90%,1111.37,A,A5,City of Los Angeles,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
185,NaN,35000.0,35000.0,35000.00000,36 months,10.65%,1140.07,B,B2,baylor college of medicine,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
268,NaN,35000.0,26825.0,26800.00000,60 months,18.64%,690.56,E,E1,"Navistar, Inc.",...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
276,NaN,35000.0,35000.0,35000.00000,36 months,8.90%,1111.37,A,A5,Twitch LLC,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,35000.0,23250.0,23200.00000,60 months,20.30%,619.88,E,E5,Prospect Transportation Inc.,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
336,NaN,35000.0,23150.0,21739.74813,60 months,17.27%,578.71,D,D3,Fed Govt-Environmental Protection Agency,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
387,NaN,35000.0,27575.0,26043.43249,60 months,12.42%,619.26,B,B4,United States Air Force,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,35000.0,35000.0,32763.05520,60 months,19.42%,916.03,E,E3,"Skanska USA Civil West, Inc",...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
496,NaN,35000.0,23250.0,23225.00000,60 months,20.30%,619.88,E,E5,The Crexent Business Centers,...,0.0,N,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
